In [2]:
import speech_recognition as sr
import math
from tqdm import tqdm
import os
import pandas as pd
import librosa
from functions import * 

In [5]:
def convert2text(audios, seconds=30, noise=False):
    r = sr.Recognizer()
    archivos = os.listdir(audios)
    datos = []

    for archivo in archivos:
        if archivo.endswith('.wav'):
            audio_path = os.path.join(audios, archivo)
            with sr.AudioFile(audio_path) as source:
                duration = source.DURATION
                splits = math.ceil(duration / seconds)
            
            transcripcion = ""  
            
            for i in tqdm(range(splits)):
                with source as src:
                    if noise:
                        r.adjust_for_ambient_noise(src)
                    audio = r.record(src, offset=i * seconds, duration=seconds)
                
                try:
                    texto = r.recognize_google(audio, language="es-MX")
                    transcripcion += texto + " "
                    speech_speed = get_speech_speed(audio_path)
                except:
                    pass
                
            datos.append([archivo, transcripcion, speech_speed])
            
    df = pd.DataFrame(datos, columns=["Archivo", "Transcripcion", "Velocidad"])
    df = df.loc[df['Transcripcion'].notna() & (df['Transcripcion'] != '')]
    df.to_csv("transcripciones.csv", index=False, encoding='latin-1')


    
  

In [6]:
def get_speech_speed(audio_path):
    # Se carga el audio
    audio, sr = librosa.load(audio_path)
            
    # Extraer segmentos del audio
    speech_segments = librosa.effects.split(audio, top_db=20)
            
    # Calcular la duración de cada segmento
    segment_durations = librosa.frames_to_time(
        [segment[1] - segment[0] for segment in speech_segments],
        sr=sr
         )
            
    # Calculate the average speed of speech
    average_speed = sum(segment_durations) / len(segment_durations)
            
    return average_speed
        

In [ ]:
%time
carpeta_audio = "C:/Users/DSTHREE/Documents/GITHUB/Speech2Text/GRABACIONES"
convert2text(carpeta_audio, seconds=10)

In [4]:
import os
import math
import speech_recognition as sr
import pandas as pd
from tqdm import tqdm
import librosa
from pyAudioAnalysis import audioSegmentation

def convertir_audio_a_texto(segmentos_audio):
    r = sr.Recognizer()
    texto_transcripcion = ""
    for segmento in segmentos_audio:
        try:
            texto = r.recognize_google(segmento, language="es-MX")
            texto_transcripcion += texto + " "
        except sr.UnknownValueError:
            print("No se pudo reconocer el audio.")
        except sr.RequestError as e:
            print("Error en la solicitud al servicio de reconocimiento de voz: {0}".format(e))
    return texto_transcripcion



def convert2text(audios, seconds=30, noise=False):
    archivos = os.listdir(audios)
    datos = []

    for archivo in archivos:
        if archivo.endswith('.wav'):
            audio_path = os.path.join(audios, archivo)
            with sr.AudioFile(audio_path) as source:
                duration = librosa.get_duration(filename=audio_path)
                splits = math.ceil(duration / seconds)

            # Obtener los segmentos de audio usando pyAudioAnalysis
            audio, sr = librosa.load(audio_path, sr=None)
            segmentos_habla = audioSegmentation.speaker_diarization(audio, sr)

            segmentos_por_persona = {}
            for segmento in segmentos_habla:
                inicio, fin = segmento[0], segmento[1]
                persona_id = segmento[2]
                if persona_id not in segmentos_por_persona:
                    segmentos_por_persona[persona_id] = []
                segmentos_por_persona[persona_id].append(audio[inicio:fin])

            # Transcribir los segmentos de cada persona
            transcripciones_por_persona = {}
            for persona_id, segmentos_persona in segmentos_por_persona.items():
                transcripcion = convertir_audio_a_texto(segmentos_persona)
                transcripciones_por_persona[persona_id] = transcripcion

            speech_speed = get_speech_speed(audio_path)

            datos.append([archivo, transcripciones_por_persona, speech_speed])

    df = pd.DataFrame(datos, columns=["Archivo", "Transcripciones", "Velocidad"])
    df.to_csv("transcripciones.csv", index=False, encoding='utf-8')

# Llamada a la función
convert2text("Z:/8. CALIDAD/INTELIGENCIA ARTIFICIAL/LLAMADAS DIMEX")


ModuleNotFoundError: No module named 'hmmlearn'